In [326]:
import numpy as np
import pandas as pd
import random
import operator as op
import requests
from bs4 import BeautifulSoup


In [325]:
# Create DataFrames for season

# Home Form/Away Form (+1 if PPG > 1.75, -1 if PPG < 1 for home and away form)

def season_df():
  h_form_dict = {"Arsenal": 1, "Aston Villa": 1, "Bournemouth": 0, "Brentford": 0, "Brighton": 1,
        "Burnley": -1, "Chelsea": 0, "Crystal Palace": 0, "Everton": -1,
        "Fulham": 0, "Liverpool": 1, "Luton": -1, "Man City": 1, "Man Utd": 1,
          "Newcastle": 1, "Nott'm Forest": 0, "Sheff Utd": -1, "Tottenham": 1, "West Ham": 0, "Wolves": 1}

  a_form_dict = {"Arsenal": 1, "Aston Villa": 0, "Bournemouth": 0, "Brentford": -1, "Brighton": 0,
        "Burnley": -1, "Chelsea": 0, "Crystal Palace": 0, "Everton": 0,
        "Fulham": -1, "Liverpool": 1, "Luton": -1, "Man City": 1, "Man Utd": 0,
          "Newcastle": 0, "Nott'm Forest": -1, "Sheff Utd": -1, "Tottenham": 0, "West Ham": 0, "Wolves": -1}
  standings = np.arange(1,21,1).tolist()
  teams = list(h_form_dict.keys())
  h_form = list(h_form_dict.values())
  a_form = list(a_form_dict.values())

  team_df = pd.DataFrame(columns = [teams, h_form, a_form]).T
  team_df.reset_index(inplace=True)
  team_df = team_df.rename(columns = {"level_0":"Team", "level_1":"Home Form", "level_2":"Away Form"})
  team_df['#'] = standings
  team_df["P"] = 0
  team_df["G"] = 0
  team_df["GC"] = 0
  team_df["+/-"] = 0
  team_df["Pts"] = 0
  team_df = team_df.set_index('#')
  team_table = team_df.drop(['Home Form','Away Form'], axis = 1)

  return team_df, team_table

EPL_df, EPL_table = season_df()
EPL_table



# player_df = pd.DataFrame(columns = ['Name','Goals'])
# player_df

,Team,P,G,GC,+/-,Pts
#,,,,,,
1,Arsenal,0,0,0,0,0
2,Aston Villa,0,0,0,0,0
3,Bournemouth,0,0,0,0,0
4,Brentford,0,0,0,0,0
5,Brighton,0,0,0,0,0
6,Burnley,0,0,0,0,0
7,Chelsea,0,0,0,0,0
8,Crystal Palace,0,0,0,0,0
9,Everton,0,0,0,0,0


In [395]:
# Work on player GS porbability 

<tr class="table-placeholder">
<td class="td-skill">
<div class="table-skill">
<div class="table-skill__indicator" style="background-color: #EBECEF;"></div>
<div class="table-skill__skill-pot">
<span class="placeholder-skill__skill"></span>
<span class="placeholder-skill__pot"></span>
</div>
</div> </td>
<td>
<div aria-hidden="true" class="d-flex align-items-center">
<div class="player-image-holder">
<i class="plyr-prfl-img"><span class="player-image"></span></i>
<figure class="small-icon-image"><span class="player-country-image"></span></figure>
</div>
<div class="text">
<span class="player-name"></span>
<span class="player-sub-text"></span>
</div>
</div> </td>
<td class="m-hide age">
<span class="player-age"></span> </td>
<td class="m-hide" scope="row">
<div class="d-flex justify-center align-items-center">
<span class="team-image"></span>
<span class="team-name"></span>
</div> </td>
<td class="text-center">
<div class="d-flex justify-content-center">
<span class="player-value-placeh

In [324]:
# Berger Table for 20 Teams
def fixture_generator():
    berger_fixtures_dict = {
    1: {1: [1, 20],	2: [2, 19],	3: [3, 18],	4: [4, 17],	5: [5, 16],	6: [6, 15],	7: [7, 14],	8: [8, 13],	9: [9, 12],	10: [10, 11]},
    2: {1: [20, 11], 2: [12, 10], 3: [13, 9], 4: [14, 8], 5: [15, 7], 6: [16, 6],7: [17, 5], 8: [18, 4], 9: [19, 3], 10: [1, 2]},	
    3: {1: [2, 20],	2: [3, 1], 3: [4, 19],	4: [5, 18],	5: [6, 17],	6: [7, 16],	7: [8, 15],	8: [9, 14],	9: [10, 13], 10: [11, 12]},
    4: {1: [20, 12], 2: [13, 11], 3: [14, 10],	4: [15, 9],	5: [16, 8],	6: [17, 7],	7: [18, 6],	8: [19, 5],	9: [1, 4], 10: [2, 3]},
    5: {1: [3, 20],	2: [4, 2],	3: [5, 1],	4: [6, 19],	5: [7, 18],	6: [8, 17],	7: [9, 16],	8: [10, 15], 9: [11, 14], 10: [12, 13]},
    6: {1: [20, 13], 2: [14, 12], 3: [15, 11],	4: [16, 10], 5: [17, 9], 6: [18, 8], 7: [19, 7], 8: [1, 6], 9: [2, 5], 10: [3, 4]},
    7: {1: [4, 20],	2: [5, 3],	3: [6, 2],	4: [7, 1],	5: [8, 19],	6: [9, 18],	7: [10, 17], 8: [11, 16], 9: [12, 15], 10: [13, 14]},
    8: {1: [20, 14], 2: [15, 13], 3: [16, 12],	4: [17, 11], 5: [18, 10], 6: [19, 9], 7: [1, 8], 8: [2, 7], 9: [3, 6], 10: [4, 5]},
    9: {1: [5, 20],	2: [6, 4],	3: [7, 3],	4: [8, 2],	5: [9, 1], 6: [10, 19], 7: [11, 18], 8: [12, 17], 9: [13, 16], 10: [14, 15]},
    10: {1: [20, 15], 2: [16, 14],	3: [17, 13], 4: [18, 12], 5: [19, 11], 6: [1, 10], 7: [2, 9], 8: [3, 8], 9: [4, 7], 10: [5, 6]},
    11: {1: [6, 20], 2: [7, 5],	3: [8, 4],	4: [9, 3],	5: [10, 2], 6: [11, 1],	7: [12, 19], 8: [13, 18], 9: [14, 17], 10: [15, 16]},
    12: {1: [20, 16], 2: [17, 15],	3: [18, 14], 4: [19, 13], 5: [1, 12], 6: [2, 11], 7: [3, 10], 8: [4, 9], 9: [5, 8], 10: [6, 7]},
    13: {1: [7, 20], 2: [8, 6],	3: [9, 5],	4: [10, 4],	5: [11, 3],	6: [12, 2],	7: [13, 1],	8: [14, 19], 9: [15, 18], 10: [16, 17]},
    14: {1: [20, 17], 2: [18, 16],	3: [19, 15], 4: [1, 14], 5: [2, 13], 6: [3, 12], 7: [4, 11], 8: [5, 10], 9: [6, 9], 10: [7, 8]},
    15: {1: [8, 20], 2: [9, 7],	3: [10, 6],	4: [11, 5],	5: [12, 4],	6: [13, 3],	7: [14, 2],	8: [15, 1],	9: [16, 19], 10: [17, 18]},
    16: {1: [20, 18], 2: [19, 17], 3: [1, 16], 4: [2, 15], 5: [3, 14],	6: [4, 13],	7: [5, 12],	8: [6, 11],	9: [7, 10],	10: [8, 9]},
    17: {1: [9, 20], 2: [10, 8], 3: [11, 7], 4: [12, 6], 5: [13, 5], 6: [14, 4], 7: [15, 3], 8: [16, 2], 9: [17, 1], 10: [18, 19]},
    18: {1: [20, 19], 2: [1, 18], 3: [2, 17], 4: [3, 16], 5: [4, 15], 6: [5, 14], 7: [6, 13], 8: [7, 12], 9: [8, 11], 10: [9, 10]},
    19: {1: [10, 20], 2: [11, 9], 3: [12, 8], 4: [13, 7], 5: [14, 6], 6: [15, 5], 7: [16, 4], 8: [17, 3], 9: [18, 2], 10: [19, 1]},
    20: {1: [20, 1], 2: [19, 2],	3: [18, 3],	4: [17, 4],	5: [16, 5],	6: [15, 6],	7: [14, 7],	8: [13, 8],	9: [12, 9],	10: [11, 10]},
    21: {1: [11, 20], 2: [10, 12], 3: [9, 13], 4: [8, 14], 5: [7, 15], 6: [6, 16],7: [5, 17], 8: [4, 18], 9: [3, 19], 10: [2, 1]},	
    22: {1: [20, 2],	2: [1, 3], 3: [19, 4],	4: [18, 5],	5: [17, 6],	6: [16, 7],	7: [15, 8],	8: [14, 9],	9: [13, 10], 10: [12, 11]},
    23: {1: [12, 20], 2: [11, 13], 3: [10, 14],	4: [9, 15],	5: [8, 16],	6: [7, 17],	7: [6, 18],	8: [5, 19],	9: [4, 1], 10: [3, 2]},
    24: {1: [20, 3],	2: [2, 4],	3: [1, 5],	4: [19, 6],	5: [18, 7],	6: [17, 8],	7: [16, 9],	8: [15, 10], 9: [14, 11], 10: [13, 12]},
    25: {1: [13, 20], 2: [12, 14], 3: [11, 15],	4: [10, 16], 5: [9, 17], 6: [8, 18], 7: [7, 19], 8: [6, 1], 9: [5, 2], 10: [4, 3]},
    26: {1: [20, 4],	2: [3, 5],	3: [2, 6],	4: [1, 7],	5: [19, 8],	6: [18, 9],	7: [17, 10], 8: [16, 11], 9: [15, 12], 10: [14, 13]},
    27: {1: [14, 20], 2: [13, 15], 3: [12, 16],	4: [11, 17], 5: [10, 18], 6: [9, 19], 7: [8, 1], 8: [7, 2], 9: [6, 3], 10: [5, 4]},
    28: {1: [20, 5],	2: [4, 6],	3: [3, 7],	4: [2, 8],	5: [1, 9], 6: [19, 10], 7: [18, 11], 8: [17, 12], 9: [16, 13], 10: [15, 14]},
    29: {1: [15, 20], 2: [14, 16],	3: [13, 17], 4: [12, 18], 5: [11, 19], 6: [10, 1], 7: [9, 2], 8: [8, 3], 9: [7, 4], 10: [6, 5]},
    30: {1: [20, 6], 2: [5, 7],	3: [4, 8],	4: [3, 9],	5: [2, 10], 6: [1, 11],	7: [19, 12], 8: [18, 13], 9: [17, 14], 10: [16, 15]},
    31: {1: [16, 20], 2: [15, 17],	3: [14, 18], 4: [13, 19], 5: [12, 1], 6: [11, 2], 7: [10, 3], 8: [9, 4], 9: [8, 5], 10: [7, 6]},
    32: {1: [20, 7], 2: [6, 8],	3: [5, 9],	4: [4, 10],	5: [3, 11],	6: [2, 12],	7: [1, 13],	8: [19, 14], 9: [18, 15], 10: [17, 16]},
    33: {1: [17, 20], 2: [16, 18],	3: [15, 19], 4: [14, 1], 5: [13, 2], 6: [12, 3], 7: [11, 4], 8: [10, 5], 9: [9, 6], 10: [8, 7]},
    34: {1: [20, 8], 2: [7, 9],	3: [6, 10],	4: [5, 11],	5: [4, 12],	6: [3, 13],	7: [2, 14],	8: [1, 15],	9: [19, 16], 10: [18, 17]},
    35: {1: [18, 20], 2: [17, 19], 3: [16, 1], 4: [15, 2], 5: [14, 3],	6: [13, 4],	7: [12, 5],	8: [11, 6],	9: [10, 7],	10: [9, 8]},
    36: {1: [20, 9], 2: [8, 10], 3: [7, 11], 4: [6, 12], 5: [5, 13], 6: [4, 14], 7: [3, 15], 8: [2, 16], 9: [1, 17], 10: [19, 18]},
    37: {1: [19, 20], 2: [18, 1], 3: [17, 2], 4: [16, 3], 5: [15, 4], 6: [14, 5], 7: [13, 6], 8: [12, 7], 9: [11, 8], 10: [10, 9]},
    38: {1: [20, 10], 2: [9, 11], 3: [8, 12], 4: [7, 13], 5: [6, 14], 6: [5, 15], 7: [4, 16], 8: [3, 17], 9: [2, 18], 10: [1, 19]}
    }


    number_allocator = random.sample(range(1,21), 20)
    team_fixture_allocator_dict = dict(zip(number_allocator, teams))

    return berger_fixtures_dict, team_fixture_allocator_dict

berger_fixtures_dict, team_id_dict = fixture_generator()

# home_list = []
# away_list = []
# for i in range (1,39,1):
#     for j in range (1,11,1):
#         home_list.append(berger_fixtures_dict[i][j][0])
#         away_list.append(berger_fixtures_dict[i][j][1])

# x = 2
# print(f"{x} has occurred {op.countOf(away_list, x)} times")
    
# x[1][1][0]



In [176]:


key_h = list(Team_dict_str_h.keys())
key_a = list(Team_dict_str_a.keys())
my_list = Team_dict_str_h[key_h[0]]
my_list


match_1 = [key_h[0],key_a[0]]
match_2 = [key_h[1],key_a[1]]

gameweek = [match_1,match_2]
goal_count = random.randint(0,8)  # Look at quartiles for suitable parameters 
print(goal_count)


while goal_count > 0:

    i = random.randint(0,1)
    home_att = random.randint(0,5)
    away_att = random.randint(0,5)
            
    Home_Score = Team_dict_str_h[key_h[i]] + home_att
    Away_Score = Team_dict_str_a[key_a[i]] + away_att
                                 
    if Home_Score - Away_Score >= 3:
        Team_dict_score_h[key_h[i]] +=1
        goal_count -=1
        # print(f"Goal!!! {Team_dict_str_h.keys[i]} {Team_dict_score_h[i]}-{Team_dict_score_a[i]} {Team_dict_str_a.keys[i]} [{Home_Score},{Away_Score}]")
    elif Away_Score - Home_Score >= 3:
        Team_dict_score_a[key_a[i]] +=1
        goal_count -=1
        # print(f"Goall!! {Team_dict_str_h.keys[i]} {Team_dict_score_h[i]}-{Team_dict_score_a[i]} {Team_dict_str_a.keys[i]}[{Home_Score},{Away_Score}]")
    else:
        continue
            # print("Nothing happened")
    
print(f"Final Score: {key_h[0]} {Team_dict_score_h[key_h[0]]}-{Team_dict_score_a[key_a[0]]} {key_a[0]}")
print(f"Final Score: {key_h[1]} {Team_dict_score_h[key_h[1]]}-{Team_dict_score_a[key_a[1]]} {key_a[1]}")



6
Final Score: Manchester United 1-1 West Ham
Final Score: Brighton 1-3 Brentford
